In [1]:
import torch
import torch.nn as nn
import pickle
import requests
import numpy as np
from scipy.stats import spearmanr

import sys
import os
sys.path.append(os.path.abspath('../..'))
from app.classes.skipgram_neg_model import SkipgramNeg

In [2]:
with open('../../app/models/skipgram_neg/skipgram_neg.pkl', 'rb') as f:
    model = pickle.load(f)

with open('../../app/models/skipgram_neg/skipgram_neg_word2index.pkl', 'rb') as f:
    word2index = pickle.load(f)

with open('../../app/models/skipgram_neg/skipgram_neg_index2word.pkl', 'rb') as f:
    index2word = pickle.load(f)

center_embeddings = model.embedding_center.weight.data
outside_embeddings = model.embedding_outside.weight.data

In [3]:
def get_embed(word):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
        
    word = torch.LongTensor([word2index[word]])
    
    embed_c = model.embedding_center(word)
    embed_o = model.embedding_outside(word)
    embed   = (embed_c + embed_o) / 2
    
    return embed[0][0].item(), embed[0][1].item()

In [4]:
#more formally is to divide by its norm
def get_dot_product(A, B):
    dot_product = np.dot(A, B)
    return dot_product

In [5]:
with open('../helper/wordsim_similarity_goldstandard.txt', 'r') as f:
    content = f.readlines()

lines = [line.strip().replace('\t', ' ').split() for line in content]

In [6]:
similarity = []

for line in lines:
    word1 = line[0]
    word2 = line[1]
    score = float(line[2])

    try:
        similarity.append(get_dot_product(get_embed(word1), get_embed(word2)))
    except:
        similarity.append(0.0)

# find the Spearman Correlation
spearman_correlation = spearmanr(similarity, [line[2] for line in lines])
print(spearman_correlation)

# find the MSE
squared_error = [(similarity[i] - float(lines[i][2]))**2 for i in range(len(similarity))]
mse = sum(squared_error) / len(similarity)
print(mse)

SignificanceResult(statistic=-0.05912540751272472, pvalue=0.4020651027047055)
33.39268596678153
